In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from config import password

### Store CSV --Top 50 School Districts-- into DataFrame

In [2]:
csv_file = "./top50_school_district_clean.csv"
district_df = pd.read_csv(csv_file).iloc[:,1:]

district_df.head()

,name,rating,city,niche_grade,number_school,number_student
0,Adlai E. Stevenson High School District No. 125,#1 Best School Districts in Chicago Area,"LINCOLNSHIRE, IL",grade A+,2,4271
1,Community High School District 128,#2 Best School Districts in Chicago Area,"VERNON HILLS, IL",grade A+,2,3287
2,New Trier Township High School District No. 203,#3 Best School Districts in Chicago Area,"NORTHFIELD, IL",grade A+,2,4040
3,Glenbrook High Schools District 225,#4 Best School Districts in Chicago Area,"GLENVIEW, IL",grade A+,4,5201
4,Township High School District No. 113,#5 Best School Districts in Chicago Area,"HIGHLAND PARK, IL",grade A+,2,3467


### Store CSV --Top 500 Elementery Schools-- into DataFrame

In [3]:
csv_file = "./top500_elementary_school_clean.csv"
school_df = pd.read_csv(csv_file).iloc[:,1:]

school_df.head()


,name,rating,district,niche_grade,stu_number,ratio
0,Meadow Glens Elementary School,#1 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,494,15:1
1,Barbara B. Rose Elementary School,#2 Best Public Elementary Schools in Chicago Area,Barrington Community Unit School District No. 220,grade A+,436,13:1
2,Grove Avenue Elementary School,#3 Best Public Elementary Schools in Chicago Area,Barrington Community Unit School District No. 220,grade A+,520,15:1
3,Ellsworth Elementary School,#4 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,258,14:1
4,Steeple Run Elementary School,#5 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,629,16:1


### Connect to local Postgres database

In [4]:
rds_connection_string = f"postgres:{password}@localhost:5432/CMAPSA"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [5]:
engine.table_names()

<ipython-input-5-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['district', 'school']

### Use pandas to load DataFrames into Postgres database tables

In [6]:
## Truncate the table
engine.execute("TRUNCATE TABLE district")
 

In [7]:
## Read Empty Table
pd.read_sql_query('select * from district', con=engine).head(100)

,id,name,rating,city,niche_grade,number_school,number_student


In [8]:
## Load DataFrame into Table
district_df.to_sql(name='district', con=engine, if_exists='append', index=False)

In [9]:
##Confirm data has been added by querying the table
pd.read_sql_query('select * from district', con=engine).head(100)

,id,name,rating,city,niche_grade,number_school,number_student
0,101,Adlai E. Stevenson High School District No. 125,#1 Best School Districts in Chicago Area,"LINCOLNSHIRE, IL",grade A+,2,4271
1,102,Community High School District 128,#2 Best School Districts in Chicago Area,"VERNON HILLS, IL",grade A+,2,3287
2,103,New Trier Township High School District No. 203,#3 Best School Districts in Chicago Area,"NORTHFIELD, IL",grade A+,2,4040
3,104,Glenbrook High Schools District 225,#4 Best School Districts in Chicago Area,"GLENVIEW, IL",grade A+,4,5201
4,105,Township High School District No. 113,#5 Best School Districts in Chicago Area,"HIGHLAND PARK, IL",grade A+,2,3467
5,106,Hinsdale Township High School District No. 86,#6 Best School Districts in Chicago Area,"HINSDALE, IL",grade A+,3,4146
6,107,Naperville Community Unit School District No. 203,#7 Best School Districts in Chicago Area,"NAPERVILLE, IL",grade A+,22,16586
7,108,Niles Township Community High School District ...,#8 Best School Districts in Chicago Area,"SKOKIE, IL",grade A+,3,4592
8,109,Barrington Community Unit School District No. 220,#9 Best School Districts in Chicago Area,"BARRINGTON, IL",grade A+,12,8557
9,110,Township High School District No. 211,#10 Best School Districts in Chicago Area,"PALATINE, IL",grade A+,7,11855


In [10]:
## Truncate the table
engine.execute("TRUNCATE TABLE school")

In [11]:
## Read Empty Table
pd.read_sql_query('select * from school', con=engine).head(100)

,id,name,rating,district,niche_grade,stu_number,ratio


In [12]:
## Load DataFrame into Table
school_df.to_sql(name='school', con=engine, if_exists='append', index=False)

In [13]:
##Confirm data has been added by querying the table
pd.read_sql_query('select * from school', con=engine).head(100)

,id,name,rating,district,niche_grade,stu_number,ratio
0,993,Meadow Glens Elementary School,#1 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,494,15:1
1,994,Barbara B. Rose Elementary School,#2 Best Public Elementary Schools in Chicago Area,Barrington Community Unit School District No. 220,grade A+,436,13:1
2,995,Grove Avenue Elementary School,#3 Best Public Elementary Schools in Chicago Area,Barrington Community Unit School District No. 220,grade A+,520,15:1
3,996,Ellsworth Elementary School,#4 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,258,14:1
4,997,Steeple Run Elementary School,#5 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,629,16:1
...,...,...,...,...,...,...,...
95,1088,Bell-Graham Elementary School,#96 Best Public Elementary Schools in Chicago ...,St. Charles Community Unit School District No....,grade A,440,15:1
96,1089,Romona Elementary School,#97 Best Public Elementary Schools in Chicago ...,Wilmette School District No. 39,grade A,532,12:1
97,1090,Bower Elementary School,#98 Best Public Elementary Schools in Chicago ...,Community Unit School District No. 200,grade A,441,12:1
98,1091,Jackson Elementary School,#99 Best Public Elementary Schools in Chicago ...,Elmhurst School District No. 205,grade A,457,14:1


### Join Top Elementery Schools and Top School Districts

In [14]:
#school_df['district']=school_df.district.str.strip(' , IL')
school_df['district']=school_df['district'].str.strip()
district_df['name']=district_df['name'].str.strip()

In [15]:
school_df['district']

0      Naperville Community Unit School District No. 203
1      Barrington Community Unit School District No. 220
2      Barrington Community Unit School District No. 220
3      Naperville Community Unit School District No. 203
4      Naperville Community Unit School District No. 203
                             ...                        
491                        Skokie School District No. 69
492                    Flossmoor School District No. 161
493                               Chicago Public Schools
494                               Chicago Public Schools
495           Downers Grove Grade School District No. 58
Name: district, Length: 496, dtype: object

In [16]:
district_df['name']

0       Adlai E. Stevenson High School District No. 125
1                    Community High School District 128
2       New Trier Township High School District No. 203
3                   Glenbrook High Schools District 225
4                 Township High School District No. 113
5         Hinsdale Township High School District No. 86
6     Naperville Community Unit School District No. 203
7     Niles Township Community High School District ...
8     Barrington Community Unit School District No. 220
9                 Township High School District No. 211
10                Township High School District No. 214
11    Indian Prairie Community Unit School District ...
12                     Elmhurst School District No. 205
13               Community High School District No. 155
14               Community Unit School District No. 200
15    Lincoln-Way Community High School District No....
16     Grayslake Community High School District No. 127
17            Consolidated High School District 

In [17]:
## Associate a School District with each school :Inner Join
school_district_lj_df = pd.merge(school_df, district_df, how='left', left_on = 'district', right_on = 'name')

In [18]:
## Display Joined DataSets
school_district_lj_df

,name_x,rating_x,district,niche_grade_x,stu_number,ratio,name_y,rating_y,city,niche_grade_y,number_school,number_student
0,Meadow Glens Elementary School,#1 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,494,15:1,Naperville Community Unit School District No. 203,#7 Best School Districts in Chicago Area,"NAPERVILLE, IL",grade A+,22.0,16586.0
1,Barbara B. Rose Elementary School,#2 Best Public Elementary Schools in Chicago Area,Barrington Community Unit School District No. 220,grade A+,436,13:1,Barrington Community Unit School District No. 220,#9 Best School Districts in Chicago Area,"BARRINGTON, IL",grade A+,12.0,8557.0
2,Grove Avenue Elementary School,#3 Best Public Elementary Schools in Chicago Area,Barrington Community Unit School District No. 220,grade A+,520,15:1,Barrington Community Unit School District No. 220,#9 Best School Districts in Chicago Area,"BARRINGTON, IL",grade A+,12.0,8557.0
3,Ellsworth Elementary School,#4 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,258,14:1,Naperville Community Unit School District No. 203,#7 Best School Districts in Chicago Area,"NAPERVILLE, IL",grade A+,22.0,16586.0
4,Steeple Run Elementary School,#5 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,629,16:1,Naperville Community Unit School District No. 203,#7 Best School Districts in Chicago Area,"NAPERVILLE, IL",grade A+,22.0,16586.0
...,...,...,...,...,...,...,...,...,...,...,...,...
491,Thomas Edison Elementary School,#495 Best Public Elementary Schools in Chicago...,Skokie School District No. 69,grade B+,513,13:1,NaN,NaN,NaN,NaN,NaN,NaN
492,Western Avenue Elementary School,#496 Best Public Elementary Schools in Chicago...,Flossmoor School District No. 161,grade B+,472,15:1,NaN,NaN,NaN,NaN,NaN,NaN
493,Canty Elementary School,#497 Best Public Elementary Schools in Chicago...,Chicago Public Schools,grade B+,786,16:1,NaN,NaN,NaN,NaN,NaN,NaN
494,Walt Disney Magnet School,#498 Best Public Elementary Schools in Chicago...,Chicago Public Schools,grade B+,1503,18:1,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
school_district_lj_df.to_csv('./top500_elementary_school_withdistrict_clean.csv')